# CS211 Data Privacy - Final Project
## Vincent Moeykens

In [1]:
# Load the data and libraries
import pandas as pd
import numpy as np
from scipy import stats
import matplotlib.pyplot as plt
import pylatex
plt.style.use('seaborn-whitegrid')

# Number of columns to include in dataset
NUM_COLUMNS = 21

def laplace_mech(v, sensitivity, epsilon):
    return v + np.random.laplace(loc=0, scale=sensitivity / epsilon)

def gaussian_mech(v, sensitivity, epsilon, delta):
    return v + np.random.normal(loc=0, scale=sensitivity * np.sqrt(2*np.log(1.25/delta)) / epsilon)

def gaussian_mech_vec(vec, sensitivity, epsilon, delta):
    return [v + np.random.normal(loc=0, scale=sensitivity * np.sqrt(2*np.log(1.25/delta)) / epsilon)
            for v in vec]

def pct_error(orig, priv):
    return np.abs(orig - priv)/orig * 100.0

cc_data = pd.read_csv('https://raw.githubusercontent.com/vmoeykens/cs211-final-project/main/data/BankChurners.csv', usecols=[x for x in range(NUM_COLUMNS)])

In [2]:
cc_data.head()

,CLIENTNUM,Attrition_Flag,Customer_Age,Gender,Dependent_count,Education_Level,Marital_Status,Income_Category,Card_Category,Months_on_book,...,Months_Inactive_12_mon,Contacts_Count_12_mon,Credit_Limit,Total_Revolving_Bal,Avg_Open_To_Buy,Total_Amt_Chng_Q4_Q1,Total_Trans_Amt,Total_Trans_Ct,Total_Ct_Chng_Q4_Q1,Avg_Utilization_Ratio
0,768805383,Existing Customer,45,M,3,High School,Married,$60K - $80K,Blue,39,...,1,3,12691.0,777,11914.0,1.335,1144,42,1.625,0.061
1,818770008,Existing Customer,49,F,5,Graduate,Single,Less than $40K,Blue,44,...,1,2,8256.0,864,7392.0,1.541,1291,33,3.714,0.105
2,713982108,Existing Customer,51,M,3,Graduate,Married,$80K - $120K,Blue,36,...,1,0,3418.0,0,3418.0,2.594,1887,20,2.333,0.000
3,769911858,Existing Customer,40,F,4,High School,Unknown,Less than $40K,Blue,34,...,4,1,3313.0,2517,796.0,1.405,1171,20,2.333,0.760
4,709106358,Existing Customer,40,M,3,Uneducated,Married,$60K - $80K,Blue,21,...,1,0,4716.0,0,4716.0,2.175,816,28,2.500,0.000


### Privacy Budget
We set our master privacy budget, $\epsilon$.

In [3]:
epsilon = 2.0

### Strategy
First we want to use the sparse vector technique and determine a clipping parameter for our data, then we want to compute a differentially private count and sum and generate an average value that is differentially private by post processing. 

In [4]:
# Here we use the above_threshold from the textbook and preserve the generality of the method
def above_threshold(queries, df, T, epsilon):
    T_hat = T + np.random.laplace(loc=0, scale = 2/epsilon)
    
    for idx, q in enumerate(queries):
        nu_i = np.random.laplace(loc=0, scale = 4/epsilon)
        if q(df) + nu_i >= T_hat:
            return idx
    return -1 # the index of the last element

def calculate_average(df, epsilon):
    partial_epsilon = epsilon / 3
    
    def create_query(b):
        return lambda df: df.clip(lower=0, upper=b).sum() - df.clip(lower=0, upper=b+1).sum()

    # Construct the stream of queries
    bs = range(1,150000,5)
    queries = [create_query(b) for b in bs]
    
    # Run AboveThreshold, using 1/3 of the privacy budget, to find a good clipping parameter
    epsilon_svt = partial_epsilon
    final_b = bs[above_threshold(queries, df, 0, epsilon_svt)]

    # Compute the noisy sum and noisy count, using 1/3 of the privacy budget for each
    epsilon_sum = partial_epsilon
    epsilon_count = partial_epsilon
    
    noisy_sum = laplace_mech(df.clip(lower=0, upper=final_b).sum(), final_b, epsilon_sum)
    noisy_count = laplace_mech(len(df), 1, epsilon_count)
    
    return noisy_sum/noisy_count

### Final Statistics to be Produced
#### Overall Averages
- Average Customer Age
- Average Months on Book
- Average Credit Limit

#### Counts
- Count of most common Income Ranges
- Count of most common Education Level

#### Averages by demographic
- Average Credit Limit of Customers <= 33y/o vs Average Credit Limit of Customers >33y/o
- Most Common Income Range of Customers with a College Degree vs Customers Without


#### First we will generate our averages

In [5]:
# Here we define a general method for computing a e-DP Average of a Field, 
# its actual value, and calculating the accuracy

def calc_avg_and_acc(df, epsilon):
    """Returns data in the form: (dp_avg, orig_avg, error)"""
    avg = sum(df) / len(df)
    dp_avg = calculate_average(df, epsilon)
    error = pct_error(avg, dp_avg)
    return (dp_avg, avg, error)

#### Next we generate our "max count" statistics

In [6]:
# We use the report_noisy_max and score methods from the textbook to set up the framework for determining
# the max counts of a parameter

def get_uniques(df, parameter):
    return df[parameter].unique()

def score(option, data, parameter):
    return len(data[data[parameter] == option])

def report_noisy_max(data, parameter, R, score, sensitivity, epsilon):
    scores = [score(r, data, parameter) for r in R]
    noisy_scores = [laplace_mech(score, sensitivity, epsilon) for score in scores]
    
    # return the argmax of the noisy scores
    max_score = np.max(noisy_scores)
    max_score_idx = noisy_scores.index(max_score)
    
    return R[max_score_idx]


In [7]:
# Here we define a general method for computing a e-DP ighest frequency count of a Field, 
# its actual value, and calculating the accuracy

def calc_max_count(data, parameter, epsilon, options):
    """Returns data in the form: (dp_avg, orig_avg)"""
    count = data[parameter].mode()[0]
    dp_count = report_noisy_max(data, parameter, options, score, 1, epsilon)
    return (dp_count, count)


#### Get all unique income and education values

In [8]:
income_options = get_uniques(cc_data, 'Income_Category')
education_options = get_uniques(cc_data, 'Education_Level')

#### Create a list of values that we consider "College Educated" for the "Education_Level" field

In [9]:
college_educated_list = ['College', 'Post-Graduate', 'Doctorate']

#### Create lists of all dp-average and dp-count values to display

In [10]:
# Create a total queries value to divide epsilon by (equally giving our privacy budget to each query)
TOTAL_QUERIES = 9

# Create a partial epsilon value to give to each of the queries so the total epsilon is 1 by sequential composition
partial_epsilon = epsilon / TOTAL_QUERIES

# Compute differentially private average statistics
dp_age_avg = calc_avg_and_acc(cc_data['Customer_Age'], partial_epsilon)
dp_months_avg = calc_avg_and_acc(cc_data['Months_on_book'], partial_epsilon)
dp_credit_limit_avg = calc_avg_and_acc(cc_data['Credit_Limit'], partial_epsilon)

# Compute differentially private count statistics
dp_income_count = calc_max_count(cc_data, 'Income_Category', partial_epsilon, income_options)
dp_education_count = calc_max_count(cc_data, 'Education_Level', partial_epsilon, education_options)

# Compute differentially private statistics about average credit limit where the customer is either <- 33y/o 
# or older than 33y/o
dp_credit_limit_less_33 = calc_avg_and_acc(cc_data[cc_data['Customer_Age'] <= 33]['Credit_Limit'], partial_epsilon)
dp_credit_limit_over_33 = calc_avg_and_acc(cc_data[cc_data['Customer_Age'] > 33]['Credit_Limit'], partial_epsilon)

# Compute differentially private statistics about most common income category for college educated and 
# non-college educated customers 
dp_income_education_count = calc_max_count(cc_data.loc[cc_data['Education_Level'].isin(college_educated_list)], 
                                           'Income_Category', partial_epsilon, income_options)
dp_income_no_education_count = calc_max_count(cc_data.loc[~cc_data['Education_Level'].isin(college_educated_list)], 
                                              'Income_Category', partial_epsilon, income_options)

# Create lists for average values and count values
averages = [dp_age_avg, dp_months_avg, dp_credit_limit_avg, dp_credit_limit_less_33, dp_credit_limit_over_33]
counts = [dp_income_count, dp_education_count, dp_income_education_count, dp_income_no_education_count]

# Display the statistics from the above lists
for avg in averages:
    print(f'Actual Average: {avg[1]}, DP Average: {avg[0]}, Percent Error: {avg[2]}')
    
for count in counts:
    print(f'Actual Max Count Value: {count[1]}, DP Max Count Value: {count[0]}')

Actual Average: 46.32596030413745, DP Average: 46.018680265272, Percent Error: 0.6632998794803315
Actual Average: 35.928409203120374, DP Average: 35.944648844825366, Percent Error: 0.04520000207407322
Actual Average: 8631.953698034848, DP Average: 8594.711839398007, Percent Error: 0.4314418257980093
Actual Average: 7212.700339558568, DP Average: 1767.1776932981477, Percent Error: 75.49908342086617
Actual Average: 8719.596833717653, DP Average: 8724.817662751331, Percent Error: 0.059874660872967486
Actual Max Count Value: Less than $40K, DP Max Count Value: Less than $40K
Actual Max Count Value: Graduate, DP Max Count Value: Graduate
Actual Max Count Value: Less than $40K, DP Max Count Value: Less than $40K
Actual Max Count Value: Less than $40K, DP Max Count Value: Less than $40K


### Generate PDF Report 
#### (If you have a LaTeX compiler installed then you can uncomment the last line to get a PDF output and not just a .tex output)

In [11]:
from pylatex import Document, Section, Subsection, Command, Math, NoEscape, MediumText, Subsubsection
from pylatex.utils import italic, NoEscape, verbatim

doc = Document()

doc.preamble.append(Command('title', 'Credit Card Data Report'))
doc.preamble.append(Command('date', NoEscape(r'\today')))
doc.append(NoEscape(r'\maketitle'))

epsilon_eq = NoEscape(r'$\epsilon$=')

doc.append('All data labeled "differentially private" in this document satisfies differential privacy for ')
doc.append(epsilon_eq)
doc.append(f'{epsilon} by sequential composition')

with doc.create(Section('Statistics')):

    with doc.create(Subsection('Basic Averages')):
        doc.append('The follwing statistics were all generated using the sparse vector technique to determine a clipping parameter for the data, and then generating differentially private sums and counts to find a differentially private average.')
        with doc.create(Subsubsection('Average Age')):
            doc.append(f'The average age of all credit card customers is {round(dp_age_avg[1], 2)}, the differentially private average age of all customers is {round(dp_age_avg[0], 2)}. This gives an error of {round(dp_age_avg[2], 4)}%.')
        with doc.create(Subsubsection('Average Months on Book')):
            doc.append(f'The average months on the book of all credit card customers is {round(dp_months_avg[1], 2)}, the differentially private average age of all customers is {round(dp_months_avg[0], 2)}. This gives an error of {round(dp_months_avg[2], 4)}%.')
        with doc.create(Subsubsection('Average Credit Limit')):
            doc.append(f'The average credit limit of all credit card customers is {round(dp_credit_limit_avg[1], 2)}, the differentially private average age of all customers is {round(dp_credit_limit_avg[0], 2)}. This gives an error of {round(dp_credit_limit_avg[2], 4)}%.')
    with doc.create(Subsection('Basic Counts')):
        doc.append('The follwing statistics were all generated using the report noisy max method to determine the highest count in a given parameter.')
        with doc.create(Subsubsection('Income Categories')):
            doc.append(f'The most common income category of all credit card customers is {dp_income_count[1]}. The most common income category as determined by using a differentially private method is {dp_income_count[0]}.')
        with doc.create(Subsubsection('Income Categories')):
            doc.append(f'The most common education level of all credit card customers is {dp_education_count[1]}. The most common education level as determined by using a differentially private method is {dp_education_count[0]}.')
    with doc.create(Subsection('Conditional Averages')):
        with doc.create(Subsubsection('Average Credit Limit for Customers 33 years old and younger.')):
            doc.append(f'The average credit limit of credit card customers who are 33 years old and younger is {round(dp_credit_limit_less_33[1], 2)}, the differentially private average is {round(dp_credit_limit_less_33[0], 2)}. This gives an error of {round(dp_credit_limit_less_33[2], 4)}%.')
        with doc.create(Subsubsection('Average Credit Limit for Customers older than 33 years old.')):
            doc.append(f'The average credit limit of credit card customers who are over 33 years old is {round(dp_credit_limit_over_33[1], 2)}, the differentially private average is {round(dp_credit_limit_over_33[0], 2)}. This gives an error of {round(dp_credit_limit_over_33[2], 4)}%.')
    with doc.create(Subsection('Conditional Counts')):
        with doc.create(Subsubsection('Most Common Income Category for College Educated Customers')):
            doc.append(f'The most common income level for college educated customers is {dp_income_education_count[1]}, the most common one calculated with a differentially private method is {dp_income_education_count[0]}. ')
        with doc.create(Subsubsection('Most Common Income Category for non College Educated Customers')):
            doc.append(f'The most common income level for non college educated customers is {dp_income_no_education_count[1]}, the most common one calculated with a differentially private method is {dp_income_no_education_count[0]}. ')


            
# Uncomment if you have a latex compiler install
#doc.generate_pdf('report', clean_tex=False)

tex = doc.dumps()
out = open("report.tex", "a")
out.write(tex)
out.close()